# Word game
https://wiki.apterous.org/Letters_game

Contestant chooses 9 letters by selecting either a vowel or a consonant until there is a total of 9 letters. The player can choose the letters in any order, but the selection must include at least 4 consonants and 3 vowels, hence there are only three valid choices in modern Countdown: 3 vowels, 6 consonants; 4 vowels, 5 consonants; and 5 vowels, 4 consonants

Distribution of letters: http://www.thecountdownpage.com/letters.htm

Steps:
1. Create the letter distribution
2. Select the letters. There must be a minimum of 3 vowels and minimum of 4 consonants. Maximum of 5 vowels and 6 consonants. 
3. Create the checker. One to check the user input. Another to find if there's a longer word. 

# Letter Pickers

In [36]:
#https://stackoverflow.com/questions/2570690/python-algorithm-to-randomly-select-a-key-based-on-proportionality-weight
import random

def weighted_pick(dic):
    r = random.uniform(0, sum(dic.values()))
    s = 0.0
    for k, w in dic.items():
        s += w
        if r < s: return k
    return k

In [37]:
def letter_classifier(st):
    vowel_count = 0
    consonant_count = 0
    for i in st:
        if i.casefold() in 'aeiou':
            vowel_count += 1
        elif i.casefold() in 'bcdfghjklmnpqrstvwxyz':
            consonant_count += 1
    return vowel_count,consonant_count

In [38]:
def letter_adder(dic):
    letter = weighted_pick(dic)
    dic[letter] -=1
    return letter

In [42]:
vowels = {"a":15,"e":21,"i":13,"o":13,"u":5}
consonants = {"b":2,"c":3,"d":6,"f":2,"g":3,"h":2,"j":1,"k":1,"l":5,"m":4,"n":8,"p":4,"q":1,"r":9,"s":9,"t":9,"v":1,"w":1,"x":1,"y":1,"z":1}
letters = []
while len(letters)<9:
    choice = input("(V)owel or (C)onsonant?")
    reqs = True
    if choice.lower() == "v":
        let = letter_adder(vowels)
        print(let)
        letters.append(let)
    elif choice.lower() == "c":
        let = letter_adder(consonants)
        print(let)
        letters.append(let)
    else:
        print("Choose a vowel by typing V, or a consonant by typing C.")
    
    letters_string = "".join(letters)
    v,c = letter_classifier(letters_string)
    
    while c == 6 and len(letters)<9:
        print("Maximum number of consonants reached. Filling with vowels.")
        let = letter_adder(vowels)
        print(let)
        letters.append(let)
    while v == 5 and len(letters)<9:
        print("Maximum number of vowels reached. Filling with consonants.")
        let = letter_adder(consonants)
        print(let)
        letters.append(let) 

print([x.upper() for x in letters])

(V)owel or (C)onsonant?v
e
(V)owel or (C)onsonant?v
a
(V)owel or (C)onsonant?v
e
(V)owel or (C)onsonant?v
e
(V)owel or (C)onsonant?v
e
Maximum number of vowels reached. Filling with consonants.
b
Maximum number of vowels reached. Filling with consonants.
t
Maximum number of vowels reached. Filling with consonants.
t
Maximum number of vowels reached. Filling with consonants.
t
['E', 'A', 'E', 'E', 'E', 'B', 'T', 'T', 'T']


# Word input

In [44]:
word = input("What is your word?")

What is your word?beat


# Word List

In [45]:
import pandas as pd

In [46]:
df = pd.read_excel("Words.xlsx", engine='openpyxl')

In [47]:
df

,Rank,Alphagram,Word(s),Length,Weighted Usefulness Score,Single Word
0,89208,IX,XI,2,0.0,XI
1,89580,IL,LI,2,0.0,LI
2,89691,HI,HI,2,0.0,HI
3,89843,IT,TI/IT,2,0.0,TI
4,90533,AA,AA,2,0.0,AA
...,...,...,...,...,...,...
104087,99903,ADEGGGIZZ,ZIGZAGGED,9,0.0,ZIGZAGGED
104088,99904,EGILOOOZZ,ZOOLOGIZE,9,0.0,ZOOLOGIZE
104089,99905,AHLOOPTYZ,ZOOPHYTAL,9,0.0,ZOOPHYTAL
104090,99906,HNOOOPTYZ,ZOOPHYTON/PHYTOZOON,9,0.0,ZOOPHYTON


# Word Checker

In [185]:
def word_check(st):
    #convert string to set in order to match alphagram
    st = st.upper()
    word = "".join(sorted(st))
    word_length = len(st)
    #shorten the length where the word is searched for
    ref = df[df["Length"] ==4]
    #does the alphagram match?
    alpha_match = (ref["Alphagram"]==word).any()
    if alpha_match:
        #sometimes there are multiple words that match the alphagram, so need to make sure that the word was spelled correctly
        ref_word = ref[(ref["Alphagram"]==word)]["Word(s)"].values[0]
        if "/" in ref_word:
            ref_words = ref_word.split("/")
            if st in ref_words:
                print(st + "is an acceptable word.")
                return True
            else:
                print(st + "is not an acceptable word.")
                return False
        elif st == ref_word:
            print(st + "is an acceptable word.")
            return True
        else:
            print(st + "is not an acceptable word.")
            return False
    else:
        print(st + "is not an acceptable word.")
        return False

In [186]:
def letter_check(st,ref):
    #need to check that the user input string uses only letters from the letter list and doesn't go over the limit
    #can't use sets because repeated letters are OK
    word_list = list(st)
    temp_letters = list(ref)
    while (len(word_list))>0:
        if word_list[0] in temp_letters:
            temp_letters.remove(word_list[0])
            word_list.remove(word_list[0])
        else:
            print("Word is not valid. "+word_list[0].upper() +" is not in the list of acceptable letters")
            return False
            break
    if len(word_list)==0:
        return True
    else:
        print(st + "does not use the correct letters")
        return False

In [187]:
letters = ['e', 'a', 'e', 'e', 'e', 'b', 't', 't', 't']

In [188]:
word

'beat'

In [190]:
letter_check("banl",letters) and word_check("banl")

Word is not valid. N is not in the list of acceptable letters


False

# Find the best word